In [65]:
import pandas as pd
import pickle
import networkx as nx

In [139]:
fname = '../data/DSR_surroundings_with_acc_on_nodes_and_edges_Full.pkl'

with open(fname, 'rb') as f:
    Gfull = pickle.load(f)

In [159]:
#attributes contained
edge_attr = ['width' ,'tunnel', 'service', 'reversed', 'v_original', 'bridge', 
            'maxspeed', 'name', 'access', 'junction', 'highway', 'oneway', 
            'u_original', 'geometry', 'accidents_list','width',  'lanes']
node_attr = ['accidents_list', 'osmid', 'highway', 'lat', 'lon']

#attributes to be removed (eg, duplicates, second order information, etc)
edge_attr2pop = ['accidents_list', 'osmid', 'geometry', 'u_original', 'v_original'] 
node_attr2pop = ['accidents_list', 'osmid', 'osmid_original', 'lat', 'lon']

#purge unwanted fields
nodeno=0
unique_attrs = set([])
for i, n in Gfull.nodes(data=True):
    nodeno += 1
    #n["nr_acc"] = len(n['accidents_list']) #this was done earlier
    for k in node_attr2pop:
        n.pop(k, None)
    unique_attrs = unique_attrs | set(n.keys())
node_attr_final  = ['index'] + list(unique_attrs)

edgeno=0
unique_attrs = set([])
for s, r, e in Gfull.edges(data=True):
    edgeno += 1
    #e["nr_acc"] = len(e['accidents_list'])
    for k in edge_attr2pop:
        e.pop(k,None)
    unique_attrs = unique_attrs | set(e.keys())
edge_attr_final  = ['Sindex', 'Rindex'] + list(unique_attrs)

In [141]:
#node_attr_final

['index', 'nr_accidents', 'y', 'street_count', 'highway', 'x']

In [142]:
#edge_attr_final

['index',
 'SRind',
 'maxspeed',
 'nr_accidents',
 'reversed',
 'oneway',
 'junction',
 'width',
 'length',
 'access',
 'bridge',
 'name',
 'service',
 'lanes',
 'highway',
 'tunnel']

In [63]:
edgeno=0
for s,r,e in Gfull.edges(data=True):
    print(f'{s} -> {r}')
    print(e)
    edgeno += 1
    if edgeno ==1:
        break

0 -> 1
{'oneway': True, 'lanes': '3', 'name': 'Klingelhöferstraße', 'highway': 'secondary', 'maxspeed': '50', 'reversed': False, 'length': 28.805905038430495, 'tunnel': [], 'service': [], 'bridge': [], 'access': [], 'junction': [], 'width': [], 'nr_acc': 0}


In [119]:
Gfull.nodes[0]

{'x': 13.352197350018844,
 'y': 52.50644985000098,
 'street_count': 0,
 'highway': []}

In [146]:
colnames=['index', 'nr_accidents', 'x', 'y', 'street_count', 'highway']
DFnodes = pd.DataFrame(data = None, columns=colnames, index=list(range(nodeno)))
nodeno=-1
for node_i, n in Gfull.nodes(data=True):
    nodeno += 1
    #print(nodeno)
    for key in colnames:
        #print(key, end=', ')
        if key == 'index':
            DFnodes[key].loc[nodeno] = node_i
        elif key in n.keys():
            if type(n[key]) == list:
                if len(n[key])==0:
                    if key=='highway':
                        DFnodes[key].loc[nodeno] = 'no_hw'
                    else:
                        raise(ValueError(f'{key}: is empty list, undefined behavior'))
            else:
                DFnodes[key].loc[nodeno] = n[key]
        #else it will be left as NA
        #print('.')
DFnodes

,index,nr_accidents,x,y,street_count,highway
0,0,9,13.352197,52.50645,0,no_hw
1,1,3,13.351931,52.506302,3,no_hw
2,410,1,13.351964,52.508369,3,no_hw
3,411,0,13.351726,52.508605,3,no_hw
4,283,0,13.351856,52.505985,3,no_hw
...,...,...,...,...,...,...
630,606,1,13.351133,52.500188,3,no_hw
631,616,1,13.351071,52.500042,3,no_hw
632,629,0,13.318173,52.510308,3,no_hw
633,630,1,13.32479,52.509917,3,no_hw


In [179]:
colnames=['Sindex', 'Rindex', 'nr_accidents', 'name', 'length','width','lanes',
          'maxspeed', 'reversed','oneway','junction','access','bridge','service','tunnel','highway']
DFedges = pd.DataFrame(data = None, columns=colnames, index=list(range(edgeno)))
edgeno=-1
for s,r,e in Gfull.edges(data=True):
    edgeno += 1
    #print(edgeno)
    for key in colnames:
        #print(key, end=', ')
        if key == 'Sindex':
            DFedges[key].loc[edgeno] = s
        elif key == 'Rindex':
            DFedges[key].loc[edgeno] = r
        elif key in e.keys():
            if type(e[key]) == list:
                if len(e[key])==0:
                    if key =='name':
                        DFedges[key].loc[edgeno] = 'unknown'
                    elif key in ['reversed', 'oneway']: 
                        DFedges[key].loc[edgeno] = False #todo: encode 1/0
                    elif key in ['bridge']:
                        DFedges[key].loc[edgeno] = 'no' #todo: encode 1/0
                    elif key in ['width','lanes','maxspeed']:
                        DFedges[key].loc[edgeno] = pd.NA
                    elif key in ['junction', 'service', 'tunnel', 'highway', 'access', 'tunnel']:
                        DFedges[key].loc[edgeno] = 'None' #todo: check again the unique list on larger map
                    else:
                        raise(ValueError(f'{key}: is empty list, undefined behavior'))
            else:
                DFedges[key].loc[edgeno] = e[key]
        #else it will be left as NA
        #print('.')
DFedges


,Sindex,Rindex,nr_accidents,name,length,width,lanes,maxspeed,reversed,oneway,junction,access,bridge,service,tunnel,highway
0,0,1,0,Klingelhöferstraße,28.805905,<NA>,3,50,False,True,None,None,no,None,None,secondary
1,0,410,2,Klingelhöferstraße,217.043305,<NA>,1,30,False,True,None,None,no,None,None,residential
2,0,411,2,Klingelhöferstraße,241.860119,<NA>,4,50,False,True,None,None,no,None,None,primary
3,1,283,0,Klingelhöferstraße,35.723,<NA>,4,50,False,True,None,None,no,None,None,primary
4,410,411,0,Klingelhöferstraße,31.291,<NA>,1,30,False,True,None,None,no,None,None,residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,606,379,1,Kleiststraße,100.86335,<NA>,2,50,False,True,None,None,no,None,None,secondary
1436,616,485,1,Kleiststraße,124.231,<NA>,NaN,50,False,True,None,None,no,None,None,secondary
1437,629,628,0,Schlüterstraße,42.422,<NA>,<NA>,30,False,False,None,None,no,None,None,residential
1438,630,521,3,Hardenbergstraße,105.152718,<NA>,2,50,False,True,None,None,no,None,None,secondary


In [180]:
DFedges.width.unique()

array([<NA>, '10.5', '10.6', '7.7', '5.9', '6', '9', '10', nan, '5.8',
       '8.5'], dtype=object)